In [1]:
from transformers import BertTokenizer, BertModel
import pandas as pd
import torch

/Users/dglandorf/opt/miniconda3/envs/llm-grammar/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_json('../dat/egpaugmented_6x100.json')

In [3]:
test_sentence = df['augmented_examples'].iloc[0][0]
test_sentence

"She's a very talented singer."

In [ ]:
# Load pre-trained model tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

: 

In [ ]:
# Encode text
input_text = test_sentence
inputs = tokenizer(input_text, return_tensors="pt")

# Get embeddings
with torch.no_grad():
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state[:,0,:].numpy()  # Taking the embedding from the first token ([CLS] token)